In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,30368
2,Huelva,Confirmados PDIA 14 días,1291
3,Huelva,Tasa PDIA 14 días,"251,57355262388683"
4,Huelva,Confirmados PDIA 7 días,592
5,Huelva,Total Confirmados,30547
6,Huelva,Curados,27853
7,Huelva,Fallecidos,377


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  30368.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7955.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 888 personas en los últimos 7 días 

Un positivo PCR cada 385 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,30368.0,592.0,1291.0,513170.0,115.361381,251.573553,127.0
Huelva-Costa,17520.0,277.0,598.0,289548.0,95.666349,206.528797,61.0
Condado-Campiña,9819.0,262.0,583.0,156231.0,167.700392,373.165377,53.0
Huelva (capital),7955.0,162.0,374.0,143837.0,112.627488,260.016547,39.0
Sierra de Huelva-Andévalo Central,2685.0,51.0,107.0,67391.0,75.677761,158.774911,13.0
Almonte,1743.0,61.0,139.0,24507.0,248.908475,567.184886,13.0
Moguer,1223.0,43.0,104.0,21867.0,196.643344,475.602506,8.0
Aljaraque,1046.0,20.0,35.0,21474.0,93.135885,162.987799,6.0
Valverde del Camino,536.0,24.0,52.0,12750.0,188.235294,407.843137,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,165.0,12.0,18.0,2211.0,542.740841,814.111262,2.0
Palma del Condado (La),1118.0,40.0,70.0,10801.0,370.336080,648.088140,3.0
Bollullos Par del Condado,950.0,33.0,87.0,14387.0,229.373740,604.712588,6.0
Almonte,1743.0,61.0,139.0,24507.0,248.908475,567.184886,13.0
Cabezas Rubias,36.0,4.0,4.0,706.0,566.572238,566.572238,0.0
Calañas,188.0,11.0,14.0,2768.0,397.398844,505.780347,3.0
Moguer,1223.0,43.0,104.0,21867.0,196.643344,475.602506,8.0
Punta Umbría,789.0,28.0,67.0,15355.0,182.351026,436.339954,5.0
Manzanilla,95.0,6.0,9.0,2118.0,283.286119,424.929178,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,481.0,1.0,9.0,3366.0,29.708853,267.379679,1.0,0.111111
Niebla,212.0,4.0,12.0,4116.0,97.181730,291.545190,1.0,0.333333
Cerro de Andévalo (El),139.0,1.0,3.0,2327.0,42.973786,128.921358,0.0,0.333333
Isla Cristina,2131.0,15.0,44.0,21393.0,70.116393,205.674753,2.0,0.340909
Nerva,173.0,4.0,11.0,5169.0,77.384407,212.807119,0.0,0.363636
San Juan del Puerto,479.0,8.0,22.0,9411.0,85.006907,233.768994,2.0,0.363636
Lepe,2112.0,9.0,24.0,27880.0,32.281205,86.083214,5.0,0.375000
Bollullos Par del Condado,950.0,33.0,87.0,14387.0,229.373740,604.712588,6.0,0.379310
Lucena del Puerto,246.0,5.0,13.0,3261.0,153.327200,398.650721,3.0,0.384615
